In [1]:
from transformers import pipeline

# Load your fine-tuned model
ner = pipeline(
    "token-classification",
    model="./recipe_ner_model",
    tokenizer="./recipe_ner_model",
    aggregation_strategy="simple"  # Group tokens
)


def clean_predictions(predictions):
    cleaned = []
    skip_next = False
    
    for i, pred in enumerate(predictions):
        if skip_next:
            skip_next = False
            continue
            
        # Merge partial tokens (like 'tbs' + '##p')
        if pred['word'].startswith('##'):
            if cleaned and i > 0:
                cleaned[-1]['word'] += pred['word'][2:]
                cleaned[-1]['end'] = pred['end']
            continue
            
        # Manual corrections
        new_pred = pred.copy()
        if pred['word'] in ['and', 'or']:
            continue
            
        if pred['word'] in ['flour', 'sugar']:
            new_pred['entity_group'] = 'INGREDIENT'
            
        if pred['word'] == 'tbs' and i < len(predictions)-1 and predictions[i+1]['word'] == '##p':
            new_pred['word'] = 'tbsp'
            new_pred['end'] = predictions[i+1]['end']
            skip_next = True
            
        cleaned.append(new_pred)
    
    return cleaned
raw_output = ner("3 cups flour and 2 tbsp sugar")
cleaned = clean_predictions(raw_output)
print(cleaned)

c:\Users\KIIT0001\AppData\Local\Programs\Python\Python311\Lib\site-packages\beartype\_util\error\utilerrwarn.py:67: BeartypeModuleUnimportableWarning: Ignoring module "onnx" importation exception:
    ImportError: DLL load failed while importing onnx_cpp2py_export: A dynamic link library (DLL) initialization routine failed.
  warn(message, cls)
c:\Users\KIIT0001\AppData\Local\Programs\Python\Python311\Lib\site-packages\beartype\_util\error\utilerrwarn.py:67: BeartypeModuleUnimportableWarning: Ignoring module "onnx" importation exception:
    ImportError: DLL load failed while importing onnx_cpp2py_export: A dynamic link library (DLL) initialization routine failed.
  warn(message, cls)


Device set to use cpu


[{'entity_group': 'AMOUNT', 'score': 0.9964065, 'word': '3', 'start': 0, 'end': 1}, {'entity_group': 'UNIT', 'score': 0.9950288, 'word': 'flour', 'start': 7, 'end': 12}, {'entity_group': 'UNIT', 'score': 0.9873297, 'word': 'and', 'start': 13, 'end': 16}, {'entity_group': 'UNIT', 'score': 0.97867626, 'word': '2', 'start': 17, 'end': 18}, {'entity_group': 'UNIT', 'score': 0.6358826, 'word': 'tbs', 'start': 19, 'end': 22}, {'entity_group': 'UNIT', 'score': 0.7788514, 'word': '##p', 'start': 22, 'end': 23}, {'entity_group': 'UNIT', 'score': 0.52295864, 'word': 'sugar', 'start': 24, 'end': 29}]
